In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=10**6,
    usecols=[  
        'user_id',
        'content_id',
        'content_type_id',
        'answered_correctly',
        'prior_question_elapsed_time',
        'prior_question_had_explanation'
    ],
       dtype={
         
                'user_id': 'int32',
                'content_id': 'int16',
                'content_type_id': 'int8',
                'answered_correctly': 'int8',
                'prior_question_elapsed_time': 'float32',
                'prior_question_had_explanation': 'boolean'
       }
    
)

In [ ]:
para=['user_id',
        'content_id',
        'content_type_id',
        'prior_question_elapsed_time',
        'prior_question_had_explanation']

In [ ]:
train_df=train_df[train_df['answered_correctly']!=-1]
train_df = train_df[train_df['content_type_id'] == 0]

In [ ]:
group_user_id=train_df.groupby('user_id')
group_content_id=train_df.groupby('content_id')


In [ ]:
user_answers_df = group_user_id.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew','var']}).copy()
user_answers_df.columns = [
    'mean_user_accuracy', 
    'questions_answered', 
    'std_user_accuracy', 
    'median_user_accuracy', 
    'skew_user_accuracy',
    'var_accuracy_user'
]
user_answers_df

In [ ]:
cont_answers_df = group_content_id.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew','var']}).copy()
cont_answers_df.columns = [
    'mean_cont_accuracy', 
    'cont_questions_answered', 
    'std_cont_accuracy', 
    'median_cont_accuracy', 
    'skew_cont_accuracy',
    'var_accuracy_content'
]

In [ ]:
train_df=train_df.merge(user_answers_df, how='left', on='user_id')
train_df=train_df.merge(cont_answers_df, how='left', on='content_id')


In [ ]:



train_df

In [ ]:
train_df = train_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
train_df=train_df.fillna(method='bfill')
train_df=train_df.fillna(method='ffill')

In [ ]:
train_df

In [ ]:
train_df['prior_question_had_explanation']=train_df['prior_question_had_explanation'].astype(int)

In [ ]:
from pandas import DataFrame
from pandas import concat


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):

    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    #if fillna:
        #agg.fillna(0)
    return agg






In [ ]:
items=['prior_question_elapsed_time', 'prior_question_had_explanation',
       'mean_user_accuracy', 'questions_answered', 'std_user_accuracy',
       'median_user_accuracy', 'skew_user_accuracy', 'var_accuracy_user',
       'mean_cont_accuracy', 'cont_questions_answered', 'std_cont_accuracy',
       'median_cont_accuracy', 'skew_cont_accuracy', 'var_accuracy_content'
    ]

In [ ]:

labels = train_df['answered_correctly']
features = train_df[items]

In [ ]:
features['prior_question_elapsed_time']=features['prior_question_elapsed_time']/features['prior_question_elapsed_time'].max()

In [ ]:

data= series_to_supervised(features,2,2)

In [ ]:
data= data.replace([np.inf, -np.inf], np.nan)

In [ ]:
data=data.fillna(method='bfill')


In [ ]:
data

In [ ]:
data=data.fillna(method='ffill')

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(data, labels, 
                                                                            test_size = 0.2, random_state = 42)

In [ ]:
train_features.shape

In [ ]:
print ( data.shape)
print(labels.shape)

In [ ]:
from lightgbm import LGBMClassifier
lgbm=LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=50,
                                learning_rate=0.1, n_estimators=400, max_depth=-1,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

In [ ]:
lgbm.fit(train_features,  train_labels)

In [ ]:
y_out=lgbm.predict_proba(test_features)

In [ ]:
#y_score=y_out[:,1]
y_score1=y_out[:,1]

In [ ]:
from sklearn.metrics import roc_curve
#fpr_clf, tpr_clf, threshold_clf=roc_curve(labels, y_score)
fpr_clf1, tpr_clf1, threshold_clf1=roc_curve(test_labels, y_score1)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [ ]:
import matplotlib.pyplot as plt
#plot_roc_curve(fpr_clf, tpr_clf,'Random Forest')
plot_roc_curve(fpr_clf1, tpr_clf1,'Random_Forest')
plt.legend(loc='bottom right')
plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score

Cval=roc_auc_score(test_labels, y_score1)
Cval

In [ ]:
dtest=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
dtest.shape

In [ ]:

    dimap=[]
    dimav=[]
    dimav.append(dtest.shape)
    dtest = dtest[dtest['content_type_id'] == 0]
    dtest=dtest.merge(user_answers_df, how='left', on='user_id')
    dtest=dtest.merge(cont_answers_df, how='left', on='content_id') 
    dtest= dtest.replace([np.inf, -np.inf], np.nan)
    dtest=dtest.fillna(method='bfill')
    dtest['prior_question_had_explanation']=dtest['prior_question_had_explanation'].astype(int)
    test= dtest[items]
    test= series_to_supervised(test,2,2)
    test=test.fillna(0)
    y_preds = lgbm.predict_proba(test)
    dtest['answered_correctly'] = y_preds[:,1]
    dimap.append(dtest.shape)
    response=dtest.loc[dtest['content_type_id'] == 0, ['row_id', 'answered_correctly']]
    

In [ ]:
import riiideducation 
env = riiideducation.make_env()
iter_test = env.iter_test()


In [ ]:
x=[]
yav=[]
yap=[]
dimap=[]
dimav=[]
ymed=[]
for (test_df, sample_prediction_d) in iter_test:
    test_df = test_df[test_df['content_type_id'] == 0]
    dimav.append(test_df.shape)
    test_d=test_df[para]
    test_d=test_d.merge(user_answers_df, how='left', on='user_id')
    test_d=test_d.merge(cont_answers_df, how='left', on='content_id') 
    ymed.append(test_d)
    test_d = test_d.replace([np.inf, -np.inf], np.nan)
    test_d=test_d.fillna(method='bfill')
    test_d=test_d.fillna(method='ffill')
    test_d['prior_question_had_explanation']=test_d['prior_question_had_explanation'].astype(int)
    test= test_d[items]
    test= series_to_supervised(test,2,2)
    
    test=test.fillna(0)
    ymed.append(test)
    y_preds = lgbm.predict_proba(test)
    x.append(y_preds[:,1])
    test_df['answered_correctly'] = y_preds[:,1]
    dimap.append(test_df.shape)
    yap.append(test_df)
    env.predict(test_df.loc[:,['row_id', 'answered_correctly']])
    

In [ ]:
dimap

In [ ]:
yap[2]